# **Core Stock Baseline Modeling - Linear Regression for GOOGL Ticker**
## In this notebook we will examine only the Google (Class C) stock for the periods we have selected for this project (01-01-2019 through 06-30-2024), and perform a Linear Regression model using the preprocessed dataframe that we created in our other notebook.  We will look to get an idea of how well our data is predicting with our new features that we created.

#### As usual let's start by bringing in our important libraries and logic for needing to complete this notebook.

In [1]:
import sys
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import cross_val_score

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)




#### Now let's read in our data that we will use for this notebook.

In [2]:
# Now let's access the main core_stock_data.csv file
csv_path = os.path.join(project_root, 'data', 'core_stock_preprocessed.csv')
preprocessed_df = pd.read_csv(csv_path, parse_dates=['Date'], index_col= 'Date')
preprocessed_df.head()

,Close,Volume,Open,High,Low,SMA_core,EMA_core,RSI_core,RMA_core,Close_Lag_1,...,EMA_Lag_Std_1_3,SMA_Lag_Avg_1_3,SMA_Lag_Std_1_3,RMA_Lag_Avg_1_3,RMA_Lag_Std_1_3,Close_Lag_Avg_1_3,Close_Lag_Std_1_3,Diff_Close_EMA_core,Ratio_Close_EMA_core,Ticker
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-14,-1.013302,-0.074793,-1.012345,-1.018762,-1.009483,-1.085271,-1.049825,1.249108,0.801504,-1.013494,...,-0.081211,-1.086158,-0.085461,0.817706,-0.625300,-1.014627,-0.348813,0.037931,0.801504,AAPL
2019-03-15,-1.007093,0.293158,-1.009876,-1.010456,-1.006383,-1.083783,-1.048013,1.325622,0.931542,-1.013494,...,-0.081211,-1.086158,-0.085461,0.817706,-0.625300,-1.014627,-0.348813,0.074966,0.931542,AAPL
2019-03-18,-1.002157,-0.011968,-1.007406,-1.007730,-1.000997,-1.081366,-1.046072,1.432516,1.018034,-1.007280,...,-0.081211,-1.085413,-0.085461,0.884049,-0.625300,-1.011515,-0.348813,0.100376,1.018034,AAPL
2019-03-19,-1.006028,0.117158,-1.000779,-1.006187,-1.000655,-1.079347,-1.044364,1.112931,0.846961,-1.002341,...,-0.066578,-1.084358,-0.062046,0.935581,-0.515015,-1.008829,-0.323448,0.054402,0.846961,AAPL
2019-03-20,-1.001793,0.102615,-1.006289,-1.004901,-1.003782,-1.077225,-1.042552,1.535368,0.916237,-1.006214,...,-0.067836,-1.082380,-0.055671,0.951042,-0.666796,-1.006398,-0.386895,0.075012,0.916237,AAPL


#### Great now let's grab the data just for our subject AAPL ticker to use in our Linear Regression model.

In [3]:
googl_data = preprocessed_df[preprocessed_df['Ticker'] == 'GOOGL']
print(googl_data.head())
googl_data.shape

               Close    Volume      Open      High       Low  SMA_core  \
Date                                                                     
2019-01-02 -0.942619 -0.446292 -0.956360 -0.946605 -0.950380  0.309924   
2019-01-03 -0.957797 -0.386258 -0.944159 -0.943791 -0.951909  0.309924   
2019-01-04 -0.930465 -0.362094 -0.948375 -0.936724 -0.944295  0.309924   
2019-01-07 -0.931582 -0.353623 -0.928409 -0.935335 -0.930749  0.309924   
2019-01-08 -0.926672 -0.425198 -0.925794 -0.929858 -0.927749  0.309924   

            EMA_core  RSI_core  RMA_core  Close_Lag_1  ...  EMA_Lag_Std_1_3  \
Date                                                   ...                    
2019-01-02 -0.937119  0.582340 -0.370774     0.416289  ...         0.018837   
2019-01-03 -0.937733 -3.408328 -0.725810    -0.942763  ...        18.172124   
2019-01-04 -0.937217  0.545261 -0.072829    -0.957951  ...        18.208224   
2019-01-07 -0.936767  0.444022 -0.110828    -0.930601  ...        -0.104597   
2019-01

(1382, 32)

#### Now let's prepare our Linear Regression model.  We will be dropping the Ticker and Close columns for the X as Ticker is an improper format and our Close feature will be our y target.  The Date feature is our index so it won't be included in the model as it will automatically be excluded from the feature set.

In [4]:
X = googl_data[['EMA_core', 'SMA_core', 'RSI_core', 'Close_Lag_1']]
y = googl_data['Close']

#### Let's now set up the rest of the model and run our first set of predictions on it.  I will be looking for the MAE (Mean Absolute Error) and the RMSE (Root Mean Squared Error) for metrics here.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

#### Now let's make our predictions and see where we end up.

In [6]:
y_pred = model.predict(X_test)

#### Let's retrieve our MAE and RMSE metrics for performance.

In [7]:
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

Mean Absolute Error (MAE): 0.020560443662807123
Root Mean Squared Error (RMSE): 0.0007427106091933438


#### Great scores, let's verify with cross validation just to make sure that our data isn't being misinterpreted by the model.

In [8]:
scores = cross_val_score(model, X, y, cv = 5, scoring = 'neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)

print(f'Cross-validated RMSE Scores: {rmse_scores}')
print(f'Mean RMSE: {rmse_scores.mean()}')

Cross-validated RMSE Scores: [0.09037801 0.02435877 0.0232167  0.04496554 0.03286379]
Mean RMSE: 0.043156562683546194


#### Our original scores were quite unrealistic, we have since tested with our model and the scores above reflect a great resting place to go forward with.  2.4 cents of error in our AAPL data in predicting future prices, not too bad!

In [9]:
correlations = X.corrwith(y)
print(correlations.sort_values(ascending = False))

Close_Lag_1    0.992437
EMA_core       0.984264
SMA_core       0.755636
RSI_core       0.048948
dtype: float64


#### There are several feature columns that are influencing our Close target because they are correlating too well.  Let's go back to our X and y assignment at the top of our notebook and drop some more of those columns to balance our score.

#### Now that our predictions look good let's make a few plots to visualize our results, starting with a plot that shows Actual vs Predicted values.

In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = y_test, y = y_pred,
                        mode = 'markers',
                        name = 'Predicted vs Actual'))

fig.add_trace(go.Scatter(x = y_test, y = y_test,
                        mode = 'lines',
                        name = 'Ideal Line'))

fig.update_layout(title = 'AAPL Actual vs Predicted Closing Prices',
                xaxis_title = 'Actual Closing Price',
                yaxis_title = 'Predicted Closing Price',
                template = 'plotly_dark')

fig.show()




#### Pretty good, we will improve this further later on in the project.  Let's look at a residuals plot as well now.

In [11]:
# Calculate the residuals first.
residuals = y_test - y_pred

# Create the residuals plot
fig = go.Figure()

fig.add_trace(go.Scatter(x = y_pred, y = residuals,
                mode = 'markers',
                name = 'Residuals'))

fig.add_trace(go.Scatter(x = y_pred, y = np.zeros_like(y_pred),
                mode = 'lines',
                name = 'Zero Line'))

fig.update_layout(title = 'Residuals vs Predicted Closing Prices',
                xaxis_title = 'Predicted Closing Prices',
                yaxis_title = 'Residuals',
                template = 'plotly_dark')

fig.show()




#### In the plot above, contrary to many other plots we create, we are wanting to see randomness.  We are looking for the data to be scattered around the zero line and randomly distributed, and that the spread is fairly symmetrical around the zero.  If there was clustering it could indicate bias in the predictions.  

#### Since we have our residuals variable established let's make a distribution plot for it to get another look.

In [12]:
fig = px.histogram(residuals, nbins = 30, marginal = 'box', histnorm = 'probability density')

fig.update_layout(title = 'AAPL Distribution of Residuals',
                xaxis_title = 'Residuals',
                yaxis_title = 'Density',
                template = 'plotly_dark')

fig.show()

#### Overall this is a good distribution.  The distribution is mostly centered around 0, and suggests that on average the model's predictions are unbiased.  The histogram appears somewhat bell-shaped though it is definitely not normal.  Again the highest concentration of residuals are very close to the 0 marker, indicating that most predictions are very accurate.  Speaking to the boxplot at the top, is shows a few outliers on each end.  The majority of the data points within the IQR (Interquartile Range) suggest that most predictions are fairly close to the actual values.  If anything is to be improved I can look at the outliers here and remove them so as the model isn't hindered by their bias.

## Summary of the Analysis on the Google (Class C) Linear Regression Model

#### In this notebook we developed a baseline Linear Regression model to predict the closing prices of GOOGL stock.  We implemented several of our previous preprocessing steps, including feature creation and scaling, followed by model creation and evaluation.

### Key Insights

#### - The model performed well as a baseline, with a Mean Squared Error (RMSE) of approximately 4.3 cents.
#### - The residuals analysis indicated that the model's errors are unbiased and normally distributed, suggesting that the model is generally reliable.
#### - The distribution of residuals showed no significant skewness or outliers, further confirming the model's robustness for a baseline.

### Potential Next Steps

#### - In the next phase, we will explore more advanced models such as Decision Trees, GRU, and Transformer models to improve prediction accuracy.
#### - We will also consider incorporating additional data and refining features based on the insights gained from this baseline model.